In [ ]:
!pip install torch-geometric -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install torch
!pip install rdflib
!pip install memory_profiler

Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
     |████████████████████████████████| 225kB 7.2MB/s 
     |████████████████████████████████| 235kB 12.4MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-1.7.2-cp37-none-any.whl size=388143 sha256=e349cf15723a09020b34debcc7515cb94aa6711a97bf0c8e503515066122026a
  Stored in directory: /root/.cache/pip/wheels/e7/66/5b/ad17ef7f04b7c425dc6930daac160c3747231b0d65f9ac7972
Successfully built torch-geometric
Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
     |████████████████████████████████| 931kB 7.2MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
     |████████████████████████████████| 389kB 8.6MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
     |████████████████████████████████| 2.6MB 8.1MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-

In [ ]:
%load_ext memory_profiler

In [ ]:
import torch 
print(torch.__version__)

1.9.0+cu102


In [ ]:
import argparse
import os.path as osp
import datetime
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Entities
from torch_geometric.utils import k_hop_subgraph
from torch_geometric.nn import RGCNConv, FastRGCNConv
from __future__ import print_function, division
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import os
import copy
from torch import save
from torch import load


In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/rdf stuff/return_vals/bgs/raw/rbgs.csv')
df['split'] = np.random.randn(df.shape[0], 1)

msk = np.random.rand(len(df)) <= 0.8

train = df[msk].drop(columns= ['split'])
test = df[~msk].drop(columns= ['split'])

In [ ]:
train.to_csv('/content/drive/MyDrive/rdf stuff/return_vals/bgs/raw/bgs_train.csv')
test.to_csv('/content/drive/MyDrive/rdf stuff/return_vals/bgs/raw/bgs_test.csv')

In [ ]:
from typing import Optional, Callable, List

import os
import os.path as osp
from collections import Counter

import gzip
import pandas as pd
import numpy as np
import torch

from torch_geometric.data import (InMemoryDataset, Data, download_url,
                                  extract_tar)

In [ ]:
class MyOwnDataSet(InMemoryDataset):

    """
    Args:
        root (string): Root directory where the dataset should be saved.
        name (string): The name of the dataset (:obj:`"AIFB"`,
            :obj:`"MUTAG"`, :obj:`"BGS"`, :obj:`"AM"`).
        transform (callable, optional): A function/transform that takes in an
            :obj:`torch_geometric.data.Data` object and returns a transformed
            version. The data object will be transformed before every access.
            (default: :obj:`None`)
        pre_transform (callable, optional): A function/transform that takes in
            an :obj:`torch_geometric.data.Data` object and returns a
            transformed version. The data object will be transformed before
            being saved to disk. (default: :obj:`None`)
    """

    def __init__(self, root, transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_dir(self) -> str:
        return osp.join(self.root, 'raw')

    @property
    def processed_dir(self) -> str:
        return osp.join(self.root, 'processed')

    @property
    def num_relations(self) -> str:
        return self.data.edge_type.max().item() + 1

    @property
    def num_classes(self) -> str:
        return self.data.train_y.max().item() + 1

    @property
    def raw_file_names(self):
        return [
            'bgs_summarized.nt',
            'bgs.csv',
            'bgs_train.csv',
            'bgs_test.csv',
        ]

    @property
    def processed_file_names(self) -> str:
        return 'data.pt'

    # def download(self):
    #     path = download_url( '/content/return_vals', self.root)

    def process(self):
        import rdflib as rdf

        graph_file, task_file, train_file, test_file = self.raw_paths

        g = rdf.Graph()
        with open(graph_file, 'rb') as f:
            g.parse(file=f, format='nt')

        freq_ = Counter(g.predicates())

        def freq(rel):
            return freq_[rel] if rel in freq_ else 0

        relations = sorted(set(g.predicates()), key=lambda rel: -freq(rel))
        subjects = set(g.subjects())
        objects = set(g.objects())
        nodes = list(subjects.union(objects))

        relations_dict = {rel: i for i, rel in enumerate(list(relations))}
        nodes_dict = {node: i for i, node in enumerate(nodes)}

        edge_list = []
        for s, p, o in g.triples((None, None, None)):
            src, dst, rel = nodes_dict[s], nodes_dict[o], relations_dict[p]
            edge_list.append([src, dst, 2 * rel])
            edge_list.append([dst, src, 2 * rel + 1])

        edge_list = sorted(edge_list, key=lambda x: (x[0], x[1], x[2]))
        edge = torch.tensor(edge_list, dtype=torch.long).t().contiguous()
        edge_index, edge_type = edge[:2], edge[2]

        labels_df = pd.read_csv(task_file)
        labels_set = set(labels_df['object'].values.tolist())
        labels_dict = {lab: i for i, lab in enumerate(list(labels_set))}
        nodes_dict = {np.unicode(key): val for key, val in nodes_dict.items()}

        train_labels_df = pd.read_csv(train_file)
        train_indices, train_labels = [], []
        for nod, lab in zip(train_labels_df['subject'].values,
                            train_labels_df['object'].values):
            train_indices.append(nodes_dict[nod])
            train_labels.append(labels_dict[lab])

        train_idx = torch.tensor(train_indices, dtype=torch.long)
        train_y = torch.tensor(train_labels, dtype=torch.long)

        test_labels_df = pd.read_csv(test_file)
        test_indices, test_labels = [], []
        for nod, lab in zip(test_labels_df['subject'].values,
                            test_labels_df['object'].values):
            test_indices.append(nodes_dict[nod])
            test_labels.append(labels_dict[lab])

        test_idx = torch.tensor(test_indices, dtype=torch.long)
        test_y = torch.tensor(test_labels, dtype=torch.long)

        data = Data(edge_index=edge_index)
        data.edge_type = edge_type
        data.train_idx = train_idx
        data.train_y = train_y
        data.test_idx = test_idx
        data.test_y = test_y
        data.num_nodes = edge_index.max().item() + 1

        data, slices = self.collate([data])
        torch.save((data, slices), self.processed_paths[0])

    # def __repr__(self) -> str:
    #     return f'{self.name.upper()}{self.__class__.__name__}()'

In [ ]:
dataset = MyOwnDataSet('/content/drive/MyDrive/rdf stuff/return_vals/bgs')
data = dataset[0]


# test_10 = []
# for i in range(10):
node_idx = torch.cat([data.train_idx, data.test_idx], dim=0)
node_idx, edge_index, mapping, edge_mask = k_hop_subgraph(node_idx, 2, data.edge_index, relabel_nodes=True)

data.num_nodes = node_idx.size(0)
data.edge_index = edge_index
data.edge_type = data.edge_type[edge_mask]
data.train_idx = mapping[:data.train_idx.size(0)]
data.test_idx = mapping[data.train_idx.size(0):]


class Net(torch.nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = RGCNConv(data.num_nodes, 16, dataset.num_relations,
                            num_bases=None)
    self.conv2 = RGCNConv(16, dataset.num_classes, dataset.num_relations,
                            num_bases=None)

  def forward(self, edge_index, edge_type):
    x = F.relu(self.conv1(None, edge_index, edge_type))
    x = self.conv2(x, edge_index, edge_type)
    return F.log_softmax(x, dim=1)


device = torch.device('cuda')
# device = torch.device('cpu') if args.dataset == 'AM' else device
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0005)


def train():
  model.train()
  optimizer.zero_grad()
  out = model(data.edge_index, data.edge_type)
  loss = F.nll_loss(out[data.train_idx], data.train_y)
  loss.backward()
  optimizer.step()
  return loss.item()


@torch.no_grad()
def test():
  model.eval()
  pred = model(data.edge_index, data.edge_type).argmax(dim=-1)
  train_acc = pred[data.train_idx].eq(data.train_y).to(torch.float).mean()
  test_acc = pred[data.test_idx].eq(data.test_y).to(torch.float).mean()
  return train_acc.item(), test_acc.item()

# print(device)
tests = []
train_start = datetime.datetime.now()
for epoch in range(1, 51):
  start = datetime.datetime.now()
  loss = train()
  train_acc, test_acc = test()
  end = datetime.datetime.now()
  time =end - start
  print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Test_acc: {test_acc:.4f}, Time: {str(time)}')
  tests.append(test_acc)
train_end = datetime.datetime.now()
train_time = train_end - train_start
print('\n Training: ' + str(train_time))
  # test_10.append(np.mean(tests))
  # print(test_10)

Epoch: 01, Loss: 2.4690, Test_acc: 0.0000, Time: 0:00:00.362326
Epoch: 02, Loss: 19.4369, Test_acc: 0.3333, Time: 0:00:00.165364
Epoch: 03, Loss: 1.6610, Test_acc: 0.6667, Time: 0:00:00.172155
Epoch: 04, Loss: 1.7373, Test_acc: 0.6667, Time: 0:00:00.164965
Epoch: 05, Loss: 2.7835, Test_acc: 0.6667, Time: 0:00:00.173559
Epoch: 06, Loss: 3.4771, Test_acc: 0.6667, Time: 0:00:00.176199
Epoch: 07, Loss: 3.8644, Test_acc: 0.6667, Time: 0:00:00.170045
Epoch: 08, Loss: 3.9824, Test_acc: 0.6667, Time: 0:00:00.166554
Epoch: 09, Loss: 3.8898, Test_acc: 0.6667, Time: 0:00:00.171596
Epoch: 10, Loss: 3.6385, Test_acc: 0.6667, Time: 0:00:00.171951
Epoch: 11, Loss: 3.2687, Test_acc: 0.6667, Time: 0:00:00.165902
Epoch: 12, Loss: 2.9806, Test_acc: 0.6667, Time: 0:00:00.169907
Epoch: 13, Loss: 2.6460, Test_acc: 0.6667, Time: 0:00:00.167521
Epoch: 14, Loss: 2.2646, Test_acc: 0.6667, Time: 0:00:00.161827
Epoch: 15, Loss: 1.8680, Test_acc: 0.6667, Time: 0:00:00.171419
Epoch: 16, Loss: 1.5081, Test_acc: 0.66

In [ ]:
np.mean(tests)

0.6466666859388351

In [ ]:
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

Model's state_dict:
conv1.weight 	 torch.Size([78, 4, 16])
conv1.root 	 torch.Size([4, 16])
conv1.bias 	 torch.Size([16])
conv2.weight 	 torch.Size([78, 16, 4])
conv2.root 	 torch.Size([16, 4])
conv2.bias 	 torch.Size([4])


In [ ]:
dataset = Entities('/temp/data', 'BGS')
data = dataset[0]

# BGS and AM graphs are too big to process them in a full-batch fashion.
# Since our model does only make use of a rather small receptive field, we
# filter the graph to only contain the nodes that are at most 2-hop neighbors
# away from any training/test node.


# test_10 = []
# for i in range(10):
node_idx = torch.cat([data.train_idx, data.test_idx], dim=0)
node_idx, edge_index, mapping, edge_mask = k_hop_subgraph(node_idx, 2, data.edge_index, relabel_nodes=True)

data.num_nodes = node_idx.size(0)
data.edge_index = edge_index
data.edge_type = data.edge_type[edge_mask]
data.train_idx = mapping[:data.train_idx.size(0)]
data.test_idx = mapping[data.train_idx.size(0):]


class Net(torch.nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = RGCNConv(data.num_nodes, 16, dataset.num_relations,
                            num_bases=None)
    self.conv2 = RGCNConv(16, dataset.num_classes, dataset.num_relations,
                            num_bases=None)

  def forward(self, edge_index, edge_type):
    x = F.relu(self.conv1(None, edge_index, edge_type))
    x = self.conv2(x, edge_index, edge_type)
    return F.log_softmax(x, dim=1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu') if args.dataset == 'AM' else device
model2, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model2.parameters(), lr=0.01, weight_decay=0.0005)



Extracting /temp/data/bgs.tgz
Processing...


http://data.bgs.ac.uk/id/EarthMaterialClass/RockName/+^PYSD does not look like a valid URI, trying to serialize this will break.
http://data.bgs.ac.uk/id/EarthMaterialClass/RockName/+^PYSD does not look like a valid URI, trying to serialize this will break.
http://data.bgs.ac.uk/id/EarthMaterialClass/RockName/+#^PRS does not look like a valid URI, trying to serialize this will break.
http://data.bgs.ac.uk/id/EarthMaterialClass/RockName/+#^PRS does not look like a valid URI, trying to serialize this will break.
http://data.bgs.ac.uk/id/EarthMaterialClass/RockName/+#^PRS does not look like a valid URI, trying to serialize this will break.
http://data.bgs.ac.uk/id/EarthMaterialClass/RockName/+#^RSR does not look like a valid URI, trying to serialize this will break.
http://data.bgs.ac.uk/id/EarthMaterialClass/RockName/+#^PRS does not look like a valid URI, trying to serialize this will break.
http://data.bgs.ac.uk/id/EarthMaterialClass/RockName/+#^RSR does not look like a valid URI, tryin

Done!


In [ ]:
dicti1 = model.conv1.state_dict()

In [ ]:
for param in model2.conv1.parameters():
  print(param.shape)

torch.Size([206, 86145, 16])
torch.Size([86145, 16])
torch.Size([16])


In [ ]:
out_w = F.pad(dicti1['weight'], (0, 0 , 43070, 43071 , 64, 64 ) ,"constant", 0)

In [ ]:
out_r = F.pad(dicti1['root'], (0, 0 , 43070 , 43071) ,"constant", 0)

In [ ]:
dicti1['weight'] =out_w
dicti1['root'] = out_r

In [ ]:
def train():
  model2.train()
  optimizer.zero_grad()
  out = model2(data.edge_index, data.edge_type)
  loss = F.nll_loss(out[data.train_idx], data.train_y)
  loss.backward()
  optimizer.step()
  return loss.item()


@torch.no_grad()
def test():
  model2.conv1.load_state_dict(dicti1)
  model2.eval()
  pred = model2(data.edge_index, data.edge_type).argmax(dim=-1)
  train_acc = pred[data.train_idx].eq(data.train_y).to(torch.float).mean()
  test_acc = pred[data.test_idx].eq(data.test_y).to(torch.float).mean()
  return train_acc.item(), test_acc.item()

# print(device)

tests = []
train_start = datetime.datetime.now()
for epoch in range(1, 51):
  start = datetime.datetime.now()
  loss = train()
  train_acc, test_acc = test()
  end = datetime.datetime.now()
  time =end - start
  print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Test_acc: {test_acc:.4f}, Time: {str(time)}')
  tests.append(test_acc)
train_end = datetime.datetime.now()
train_time = train_end - train_start
print('\n Training: ' + str(train_time))


In [ ]:
np.mean(tests)